<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Plugin

**Description:** This notebook gets the list of LinkedIn profile that interacted with a post, determine if they fit the ICP using prompt engineering, and sends a report via email with a CTA that opens a Chat Agent to explore outreach possibilities.

## Input

### Import libraries

In [ ]:
import os
import json
import pandas as pd

import naas 
import naas_drivers
import openai

from naas_drivers import emailbuilder, linkedin
from datetime import datetime, date
import random
import time
from dateutil.parser import parse
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import plotly.io as pio
try:
    import tiktoken
except:
    !pip install tiktoken --user
    import tiktoken

### Setup variables
<a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [ ]:
# Define scenario 
is_demo = True # Default to True

# For Emails
EMAIL_TO = "jeremy@naas.ai"  # you will receive weekly summary at this email
EMAIL_FROM = None  # summary will have this email as sender. Only available for your naas email, otherwise you will receive this email from notification@naas.ai
EMAIL_SUBJECT = (f"🌍 Open Data Engine Demo - Email Update, {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
DATE_FORMAT = "%Y-%m-%d"


# For openAI API
openai_api_key = naas.secret.get("OPENAI_API_KEY")

# For AI Chat Plugin
plugin_name = "🌍 Open Data Agent Demo"
plugin_model = "gpt-3.5-turbo-16k"
plugin_temperature = 0
plugin_max_tokens = 8192
system_prompt_max_tokens = 2084

# For Asset Generation
output_dir = "../outputs/"
csv_file_name = "data.csv"
image_file_name = "image.png"
plugin_file_name = "plugin.json"

## Model

### Setup directories

In [ ]:
# Check if directory exists and create it if not
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
# Generate outputs files path
csv_file_path = os.path.join(output_dir, csv_file_name)
image_file_path = os.path.join(output_dir, image_file_name)
plugin_file_path = os.path.join(output_dir, plugin_file_name)
print('📂 CSV file path:', csv_file_path)
print('📂 Image file path:', image_file_path)
print('📂 Plugin file path:', plugin_file_path)

### Get or load data

In [ ]:
import os
import glob
import pandas as pd

dir_path = '../outputs/'

def get_or_load_data(dir_path):
    # Initialize an empty DataFrame to hold the aggregated data
    aggregated_df = pd.DataFrame()
    
    # Search for all files ending with '_tracker.csv' in the specified directory
    csv_files = glob.glob(os.path.join(dir_path, '*_tracker.csv'))
    
    # Loop through the list of CSV files and read each one into a DataFrame
    for csv_file_path in csv_files:
        df = pd.read_csv(csv_file_path)
        
        # Concatenate the newly read DataFrame with the aggregated DataFrame
        aggregated_df = pd.concat([aggregated_df, df], ignore_index=True)
        
    return aggregated_df

# Example usage
# dir_path should be the path to the directory containing your '_tracker.csv' files
aggregated_df = get_or_load_data(dir_path)
aggregated_df

### Add weight column

In [ ]:
import pandas as pd

def add_weight_column(df, ticker_weight_map):
    """
    Add a new column 'WEIGHT' to the DataFrame based on a key-value array (dictionary) that maps 'TICKER' to 'WEIGHT'.
    
    Parameters:
    - df: Pandas DataFrame containing a column named 'INDICATOR'
    - ticker_weight_map: Dictionary mapping 'INDICATOR' to 'WEIGHT'
    
    Returns:
    - Modified DataFrame with the new 'WEIGHT' column
    """
    # Check if 'TICKER' column exists in the DataFrame
    if 'INDICATOR' not in aggregated_df.columns:
        print("Error: The DataFrame must contain a 'TICKER' column.")
        return df
    
    # Add the 'WEIGHT' column by mapping 'TICKER' values to their corresponding 'WEIGHT' values
    aggregated_df['WEIGHT'] = aggregated_df['INDICATOR'].map(ticker_weight_map)
    
    # Handle rows where 'TICKER' was not found in the key-value array (dictionary)
    aggregated_df['WEIGHT'].fillna('N/A', inplace=True)
    
    return df

# Sample DataFrame
data = aggregated_df

# Sample key-value array (dictionary) mapping 'TICKER' to 'WEIGHT'
ticker_weight_map = {'AAPL': 2, 'GOOGL': 3, 'AMZN': 4, 'MSFT': 1, 'META': 3}

# Add 'WEIGHT' column to DataFrame
df = add_weight_column(aggregated_df, ticker_weight_map)
df

### Compute Contribution

In [ ]:
import pandas as pd

# Get data
data = df

# Create a DataFrame
df = pd.DataFrame(data)

# Add the 'CONTRIBUTION' column by multiplying 'SCORE' and 'WEIGHT'
df['CONTRIBUTION'] = df['SCORE'] * df['WEIGHT']

# Display the DataFrame
df

In [ ]:
import pandas as pd

def add_uwt_row(df):
    """
    Add a new row for each unique value in the 'SCENARIO' column.
    The new row will have 'INDICATOR' set to 'UWT' and other columns aggregated or set to specific values.
    
    Parameters:
    - df: Pandas DataFrame
    
    Returns:
    - Modified DataFrame with the new rows
    """
    # Group by 'SCENARIO' and calculate the sum of 'CONTRIBUTION'
    grouped_df = df.groupby('SCENARIO').agg({'CONTRIBUTION': 'sum'}).reset_index() 
    
    #Divide the sum of 'CONTRIBUTION' by 7
   # grouped_df['CONTRIBUTION'] = grouped_df['CONTRIBUTION'] / 5
    
    # Create new rows with 'INDICATOR' set to 'UWT' and 'CONTRIBUTION' set to the sum for each 'SCENARIO'
    new_rows = grouped_df.copy()
    new_rows['INDICATOR'] = 'UWT'
    
    # Optionally, set other columns to specific values or aggregations
    new_rows['ENTITY'] = 'N/A'
    new_rows['TYPE'] = 'Aggregated'
    new_rows['SOURCE'] = 'Calculated'
    new_rows['VALUE'] = 'N/A'
    new_rows['MIX'] = 'N/A'
    new_rows['MAX'] = 'N/A'
    new_rows['SCORE'] = 'N/A'
    new_rows['WEIGHT'] = 'N/A'
    
    # Append the new rows to the original DataFrame
    df = pd.concat([df, new_rows], ignore_index=True)
    
    return df

# Sample DataFrame
data = {
    'ENTITY': [0, 0, 0, 0, 0],
    'SCENARIO': ['Universal Tracker'] * 5,
    'INDICATOR': ['GOOGL', 'MSFT', 'AMZN', 'AAPL', 'META'],
    'TYPE': ['Financial'] * 5,
    'SOURCE': ['Yahoo Finance'] * 5,
    'VALUE': ['$135.90', '$328.80', '$135.10', '$187.60', '$295.10'],
    'MIX': ['$86.00', '$222.30', '$81.80', '$125.00', '$113.90'],
    'MAX': ['$135.90', '$359.50', '$142.20', '$196.40', '$325.50'],
    'SCORE': [10.00, 7.76, 8.82, 8.77, 8.56],
    'WEIGHT': [3, 1, 4, 2, 3],
    'CONTRIBUTION': [30.00, 7.76, 35.28, 17.54, 25.68]
}

df = pd.DataFrame(data)

# Add 'UWT' rows to DataFrame
df = add_uwt_row(df)

# Display the modified DataFrame
df

### Create chart

In [ ]:
def create_vertical_barchart(df,
                               label="INDICATOR",
                               value="SCORE",
                               value_d="SCORE"):
    # Init
    fig = go.Figure()
    
    # Return empty fig if dataframe is empty
    if len(df) == 0:
        return fig
    
    # Create fig
    fig.add_trace(
        go.Bar(
            y=df[value],
            x=df[label],
            textposition="outside",
            marker=dict(color="#0f93d2"),
            orientation="v"
        )
    )
    
    # Add logo
    fig.add_layout_image(
        dict(
            #source="logo.png",
            xref="paper",
            yref="paper",
            x=0.28,
            y=.035,
            sizex=0.15,
            sizey=0.15,
            xanchor="right",
            yanchor="bottom"
        )
    )
    
    fig.update_traces(showlegend=False)
    
    # Plotly: Create title
    title = f"<b><span style='font-size: 20px;'>Daily Score per Indicator</span></b>"
    fig.update_layout(
        title=title,
        #title_x=0.09,
        title_font=dict(family="Arial", color="black"),
        paper_bgcolor="#ffffff",
        plot_bgcolor="#ffffff",
        width=1200,
        height=600,
        #margin_pad=10,
        #margin_r=10,
        #margin_l=10,
    )
    
    fig.update_xaxes(showticklabels=True)
    
    return fig

create_vertical_barchart(df)

### Create asset from chart

In [ ]:
# Create the chart
chart = create_vertical_barchart(df)

# Save as PNG
pio.write_image(chart, image_file_path)

#graph_url = naas.asset.add("chart.html", {"inline": True})
graph_image = naas.asset.add(image_file_path)

### Set email parameters

In [ ]:
today = datetime.now().strftime(DATE_FORMAT)
today

### Create Naas Chat plugin

In [ ]:
system_prompt = f"""Act as a Open Data Agent who has access to a list of indicators from financial, extra-financial data and alternative data.
Your role is to help people follow the portfolio of indicators that makes sense for them.
The first message should be about presenting yourself with a maximum of 5 bullet points and displaying the current content analytics data to be displayed as an image inside the markdown of the chat:{graph_image}.
Then, wait for the first answer from the user, and then start with the first high-level analysis.
Here is the data from the content analytics that you should focus on: {df}
"""

### Check token count 

In [ ]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

system_prompt_tokens = num_tokens_from_string(system_prompt, "cl100k_base")
if system_prompt_tokens > system_prompt_max_tokens:
    print("⚠️ Be carefull, your system prompt looks too big. Tokens:", system_prompt_tokens)
else:
    print("✅ System prompt tokens count OK:", system_prompt_tokens)

### Generate plugin

In [ ]:
# Create json
plugin = {
    "name": plugin_name,
    "model": plugin_model,
    "temperature": plugin_temperature,
    "max_tokens": plugin_max_tokens,
    "prompt": system_prompt,
}

# Save dict to JSON file
with open(plugin_file_path, "w") as f:
    json.dump(plugin, f)
print("💾 Plugin successfully saved:")

plugin = naas.asset.add(plugin_file_path, params={"inline": True})

### Generate email content

In [ ]:
def format_number(num):
    NUMBER_FORMAT = "{:,.0f}"
    num = str(NUMBER_FORMAT.format(num)).replace(",", " ")
    return num

In [ ]:
def email_brief(
    today,
):
    content = {
        'title': ("🌍 Open Data Engine - Email Update"),
        'heading': (f"Date:{today}"),
        "txt_intro": (
            f"Hi there,<br><br>" f"Here is your open data engine email as of {today}."
        ),
        "title_1": emailbuilder.text(
            "Overview", font_size="27px", text_align="center", bold=True
        ),
        "text_1": emailbuilder.text(
            f"Your indicators has been performing as follows today:"
        ),
        "image_1": emailbuilder.image(graph_image),
        "button_1": emailbuilder.button(
            link=(f"https://naas.ai/chat/use?plugin_url={plugin}"),
            text="Start Chatting With Agent",
            background_color="#181a1c",
        ),
        "footer_cs": emailbuilder.footer_company(naas=True),
    }

    email_content = emailbuilder.generate(display="iframe", **content)
    return email_content


email_content = email_brief(
    today,
)

## Output

### Send email notification

In [ ]:
# sends the email
naas.notification.send(
    email_to=EMAIL_TO, subject=EMAIL_SUBJECT, html=email_content, email_from=EMAIL_FROM
)